In [220]:
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(max_features=5000)
vectors = vectorizer.fit_transform(newsgroups_train.data)
newsgroups_train = fetch_20newsgroups(subset='train', remove=('headers', 'footers', 'quotes'))

In [221]:
print(newsgroups_train.target.shape)
vectors_train = vectors.toarray()
print(newsgroups_train.target[0])

(11314,)
7


In [178]:
print(np.count_nonzero(vectors_train[0]))

53


In [222]:
targets = []
for i in range(20):
    targets.append([])
for item in newsgroups_train.target:
    for i in range(20):
        if i == item:
            targets[i].append(1)
        else:
            targets[i].append(-1)
targets = np.array(targets)
print(targets[0])

[-1 -1 -1 ... -1 -1 -1]


In [223]:
import numpy as np
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
import math
# from data_loader import data


def sigmoid(X):
    return 1 / (1 + np.exp(-X))

class Perceptron:
    def __init__(self, input_dim, learning_rate=0.001, epochs=100):
        self.learning_rate = learning_rate
        self.random_init(input_dim, 1)
        self.hidden_dim = 1
        self.input_dim = input_dim
        self.epochs = epochs

    def random_init(self, input_dim, hidden_dim):
        """ Intialize the model parameters randomly """
        self.W1 = np.random.normal(loc=0.0, scale=0.1, size=(input_dim, hidden_dim))
        self.b1 = np.zeros((hidden_dim, 1))

    def forward_propagation(self, x):
        """
        Compute forward propagation step. This is called by fit function.
        input:
            x: a numpy vector. Single data point.
        output:
            z1: input layer pre-activation function output.
            a1: input layer activation function output.
        """

        #############################################################################
        # TODO: Compute the input layer pre-activation linear function.             #
        #############################################################################
        # START OF YOUR CODE
        z1 = np.dot(np.transpose(self.W1), x) + self.b1
        
        if z1 >= 0:
            a1 = 1
        else:
            a1 = -1

        return z1, a1

    def update(self, x, y, z1, a1):
        #############################################################################
        # TODO: Compute W2 and b2 gradiants.                                        #
        #############################################################################
        # START OF YOUR CODE
        if y != a1: 
            self.W1 = self.W1 + (np.multiply(self.learning_rate, np.multiply(y, x)))
            self.b1 = self.b1 + (self.learning_rate * y)

    def fit(self, X, Y, plot_loss=False):
        index = list(range(X.shape[0]))
        for epoch in range(self.epochs):
            np.random.shuffle(index)
            for idx in index:
                x = X[idx]
                x= x.reshape(1, self.input_dim).T
                y = Y[idx]
                z1, a1 = self.forward_propagation(x)
                self.update(x, y, z1, a1)

    def predict(self, X):
        #############################################################################
        # TODO: Implement the predict function.                                     #
        #############################################################################
        # START OF YOUR CODE
        results = []
        for x in X:
            x = x.reshape(1, self.input_dim).T
            z1, a1= self.forward_propagation(x)
            results.append([z1, a1])
        Y_hat = np.array(results).reshape([len(X), 2])
        return Y_hat


In [224]:
classifiers = []
for it in range(20):
    classifiers.append(Perceptron(5000, 1, epochs=50))
    classifiers[-1].fit(vectors_train, targets[it])

In [225]:
newsgroups_test = fetch_20newsgroups(subset='test', remove=('headers', 'footers', 'quotes'))
vectors_test = vectorizer.transform(newsgroups_test.data)
vectors_test = vectors_test.toarray()
test_targets = []
for i in range(20):
    test_targets.append([])
for item in newsgroups_test.target:
    for i in range(20):
        if i == item:
            test_targets[i].append(1)
        else:
            test_targets[i].append(0)
test_targets = np.array(test_targets)

In [226]:
metrics = []
results = []
for i in range(20):
    metrics.append({"tp":0, "fn":0, "tn":0, "fp":0})
    result = classifiers[i].predict(vectors_train)
    results.append(result)
    count = 0
    for item in result:
        if item[1] == targets[i][count] and item[1] == 1:
            metrics[-1]['tp'] += 1
        elif item[1] == targets[i][count] and item[1] == -1:
            metrics[-1]['tn'] += 1
        elif item[1] != targets[i][count] and item[1] == 1:
            metrics[-1]['fp'] += 1 
        elif item[1] != targets[i][count] and item[1] == -1:
            metrics[-1]['fn'] += 1 
        count += 1

In [227]:
metrics

[{'tp': 480, 'fn': 0, 'tn': 10261, 'fp': 573},
 {'tp': 584, 'fn': 0, 'tn': 10152, 'fp': 578},
 {'tp': 558, 'fn': 33, 'tn': 10723, 'fp': 0},
 {'tp': 574, 'fn': 16, 'tn': 10723, 'fp': 1},
 {'tp': 556, 'fn': 22, 'tn': 10736, 'fp': 0},
 {'tp': 590, 'fn': 3, 'tn': 10721, 'fp': 0},
 {'tp': 575, 'fn': 10, 'tn': 10721, 'fp': 8},
 {'tp': 594, 'fn': 0, 'tn': 10317, 'fp': 403},
 {'tp': 582, 'fn': 16, 'tn': 10716, 'fp': 0},
 {'tp': 570, 'fn': 27, 'tn': 10717, 'fp': 0},
 {'tp': 583, 'fn': 17, 'tn': 10714, 'fp': 0},
 {'tp': 574, 'fn': 21, 'tn': 10719, 'fp': 0},
 {'tp': 573, 'fn': 18, 'tn': 10720, 'fp': 3},
 {'tp': 574, 'fn': 20, 'tn': 10720, 'fp': 0},
 {'tp': 576, 'fn': 17, 'tn': 10721, 'fp': 0},
 {'tp': 590, 'fn': 9, 'tn': 10715, 'fp': 0},
 {'tp': 531, 'fn': 15, 'tn': 10767, 'fp': 1},
 {'tp': 564, 'fn': 0, 'tn': 10334, 'fp': 416},
 {'tp': 450, 'fn': 15, 'tn': 10849, 'fp': 0},
 {'tp': 359, 'fn': 18, 'tn': 10937, 'fp': 0}]

In [228]:
all_results = []
for i in range(vectors_train.shape[0]):
    data = []
    for j in range(20):
        data.append(results[j][i][0])
    all_results.append(data)
        

In [229]:
count = 0
t = 0
f = 0
for item in all_results:
    if np.argmax(item) == newsgroups_train.target[count]:
        t += 1
    else:
        f += 1
    count += 1

In [230]:
print(t)
print(f)

10917
397


In [231]:
print(t/(t+f))

0.9649107300689411


In [232]:
metrics = []
results = []
for i in range(20):
    metrics.append({"tp":0, "fn":0, "tn":0, "fp":0})
    result = classifiers[i].predict(vectors_test)
    results.append(result)
    count = 0
    for item in result:
        if item[1] == test_targets[i][count] and item[1] == 1:
            metrics[-1]['tp'] += 1
        elif item[1] == test_targets[i][count] and item[1] == -1:
            metrics[-1]['tn'] += 1
        elif item[1] != test_targets[i][count] and item[1] == 1:
            metrics[-1]['fp'] += 1 
        elif item[1] != test_targets[i][count] and item[1] == -1:
            metrics[-1]['fn'] += 1 
        count += 1
all_results = []
for i in range(vectors_test.shape[0]):
    data = []
    for j in range(20):
        data.append(results[j][i][0])
    all_results.append(data)
count = 0
t = 0
f = 0
for item in all_results:
    if np.argmax(item) == newsgroups_test.target[count]:
        t += 1
    else:
        f += 1
    count += 1
print(t/(t+f))

0.554567180031864
